# Image Deraining Project

### Imports

In [ ]:
# !pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu117
# !pip3 install numpy scipy tqdm opencv-python pandas scikit-learn matplotlib lightning

In [ ]:
# Python Packages
from torch.utils.data import DataLoader

import warnings
import torch

# Local Modules
from utilities.utils import train_lightning, plot_loss
from utilities.datasets import SID_dataset, SID_dataset_mini
from models.prenet import PReAENetPL, PReDeconvNetPL


In [ ]:
warnings.filterwarnings('ignore')
torch.manual_seed(42)

DEVICE = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')


IM_SIZE = 256

In [ ]:
input_path = "data/rain_streak/training/rainy_image/"
label_path = "data/rain_streak/training/ground_truth/"

valid_input_path = "data/rain_streak/validation/rainy_image/"
valid_label_path = "data/rain_streak/validation/ground_truth/"

batch_size = 4

dataset_train = SID_dataset_mini(input_path, label_path, IM_SIZE)
dataset_valid = SID_dataset(valid_input_path, valid_label_path, IM_SIZE)
train_loader = DataLoader(
    dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(
    dataset_valid, batch_size=batch_size, shuffle=True, pin_memory=True)


In [ ]:
net1 = PReAENetPL(DEVICE)

p_net1 = sum(p.numel() for p in net1.parameters())

print(f'PRettentiveNet(PL):\n\t\t\t\t{p_net1}', end='\n\n\n')


In [ ]:
model = [net1, 'PRettentiveNet(PL)']

train_dict = {
    'train_loader': train_loader,
    'val_loader': valid_loader,
    'epochs': 100
}


In [ ]:
print(f'Training {model[1]}')
train_lightning(model[0], **train_dict)

In [ ]:
torch.save(net1.state_dict(), 'saved1_state_AEATPL.pth')